- Sample large video:
    - url: https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/1005-2004/NewAdd.Dream.Team.8__%231_label_A.mp4
    - size: 438.6 MB
    - length: 1:34:56

In [1]:
import torchvision
import cv2
import numpy as np

torchvision.disable_beta_transforms_warning()

In [2]:
# Auto-reload module to access .py files easily
%load_ext autoreload
%autoreload 2

import os
import sys

src_path = os.path.abspath("../src/")
if not src_path in sys.path:
    sys.path.append(src_path)

import transforms as custom_transforms
import dataset_utils

In [3]:
vr_init = custom_transforms.VideoReaderInit()
clip_sample = custom_transforms.TemporalClipSample(clip_len=32, sampling_rate=2, num_clips=-1)
clips_batching = custom_transforms.ClipsBatching(batch_size=8)
vr_decode = custom_transforms.VideoDecode()

In [4]:
sample_fpath = "https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/1005-2004/NewAdd.Dream.Team.8__%231_label_A.mp4"
video_ex = {
    "meta": {
        "filename": sample_fpath
    }        
}

In [5]:
video_ex = vr_init(video_ex)
video_ex

{'meta': {'filename': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/1005-2004/NewAdd.Dream.Team.8__%231_label_A.mp4',
  'total_frames': 136704,
  'video_reader': < cv2.VideoCapture 0000019A287E6830>,
  'avg_fps': 24.0}}

In [6]:
video_ex = clip_sample(video_ex)
video_ex

{'meta': {'filename': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/1005-2004/NewAdd.Dream.Team.8__%231_label_A.mp4',
  'total_frames': 136704,
  'video_reader': < cv2.VideoCapture 0000019A287E6830>,
  'avg_fps': 24.0,
  'frame_indices': array([[     0,      2,      4, ...,     58,     60,     62],
         [    64,     66,     68, ...,    122,    124,    126],
         [   128,    130,    132, ...,    186,    188,    190],
         ...,
         [136512, 136514, 136516, ..., 136570, 136572, 136574],
         [136576, 136578, 136580, ..., 136634, 136636, 136638],
         [136640, 136642, 136644, ..., 136698, 136700, 136702]]),
  'num_clips': 2136,
  'clip_len': 32,
  'sampling_rate': 2}}

In [7]:
video_ex["meta"]["frame_indices"].shape

(2136, 32)

In [8]:
video_ex = clips_batching(video_ex)
video_ex

[{'meta': {'filename': 'https://huggingface.co/datasets/jherng/xd-violence/resolve/main/data/video/1005-2004/NewAdd.Dream.Team.8__%231_label_A.mp4',
   'total_frames': 136704,
   'video_reader': < cv2.VideoCapture 0000019A287E6830>,
   'avg_fps': 24.0,
   'clip_len': 32,
   'sampling_rate': 2,
   'batch_id': 0,
   'frame_indices': array([[  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
            26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
            52,  54,  56,  58,  60,  62],
          [ 64,  66,  68,  70,  72,  74,  76,  78,  80,  82,  84,  86,  88,
            90,  92,  94,  96,  98, 100, 102, 104, 106, 108, 110, 112, 114,
           116, 118, 120, 122, 124, 126],
          [128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152,
           154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178,
           180, 182, 184, 186, 188, 190],
          [192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216,
           2

In [9]:
video_ex[0]["meta"]["frame_indices"].shape

(8, 32)

In [10]:
video_ex0 = vr_decode(video_ex[0])

In [13]:
%%time
video_ex1 = vr_decode(video_ex[1])

CPU times: total: 2.25 s
Wall time: 1min 34s


In [15]:
video_ex["inputs"].shape

torch.Size([8, 32, 3, 480, 640])

In [8]:
container = video_ex["meta"]["video_reader"]
container

In [9]:
indices = video_ex["meta"]["frame_indices"]
indices.shape

(2136, 32)

In [10]:
clips = container.get_batch(indices).asnumpy()
clips.shape

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [12]:
clips = container.get_batch(indices.flatten()).asnumpy()
clips.shape

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [13]:
clips = container.get_batch(indices.flatten()[:1000]).asnumpy()
clips.shape

(1000, 480, 640, 3)

In [16]:
clips = container.get_batch(indices.flatten()[:1000]).asnumpy()
clips.shape

(1000, 480, 640, 3)

In [17]:
clips = container.get_batch(indices[:30]).asnumpy()
clips.shape

(960, 480, 640, 3)

In [10]:
batch_0_100 = container.get_batch(list(range(0, 100))).asnumpy()
print(batch_0_100.shape)
print(batch_0_100.mean())
print(batch_0_100.std())

(100, 480, 640, 3)
75.67996382378472
69.57179861747308


In [11]:
batch_100_200 = container.get_batch(list(range(100, 200))).asnumpy()
print(batch_100_200.shape)
print(batch_100_200.mean())
print(batch_100_200.std())

(100, 480, 640, 3)
86.23304033203125
73.85553774912123


In [12]:
del batch_0_100, batch_100_200

In [13]:
import gc
gc.collect()

2505

In [10]:
batch_0_100 = container.get_batch(list(range(0, 100))).asnumpy()
container.seek(0)
print(batch_0_100.shape)
print(batch_0_100.mean())
print(batch_0_100.std())

(100, 480, 640, 3)
75.67996382378472
69.57179861747308


In [11]:
batch_100_200 = container.get_batch(list(range(100, 200))).asnumpy()
container.seek(0)
print(batch_100_200.shape)
print(batch_100_200.mean())
print(batch_100_200.std())

(100, 480, 640, 3)
86.23304033203125
73.85553774912123


In [12]:
del batch_0_100, batch_100_200

In [13]:
import gc
gc.collect()

114

In [20]:
clips = container.get_batch([0, 1, 2]).asnumpy()

In [22]:
clips.shape

(3, 480, 640, 3)